### Prepare Dependencies

In [ ]:
!apt-get update
!apt-get install zip unzip
!git clone --branch updt https://github.com/TheLastBen/diffusers
!pip install -q diffusers
!pip install -U -r diffusers/examples/dreambooth/requirements.txt
!pip install -U accelerate
!pip install -U gdown
!pip install -U wget
!pip install -U deepspeed
!pip install -U bitsandbytes

### Important Inputs for the training:
- The name of the model from HuggingFace, Models I like to use:
    - runwayml/stable-diffusion-v1-5 
    - nitrosocke/Arcane-Diffusion
    - nitrosocke/mo-di-diffusion 
    - nitrosocke/spider-verse-diffusion
    - nitrosocke/Ghibli-Diffusion
    - nitrosocke/archer-diffusion
    - nitrosocke/classic-anim-diffusion
    - DGSpitzer/Cyberpunk-Anime-Diffusion
    - dallinmackay/Tron-Legacy-diffusion
    - Envvi/Inkpunk-Diffusion
    - dallinmackay/Van-Gogh-diffusion
    - ogkalu/Comic-Diffusion
- The class name that your images will belong to:
    - person
    - man
    - woman
- The unique token to identify your character in the prompt

In [ ]:
MODEL_NAME = '' #nitrosocke/Arcane-Diffusion - nitrosocke/mo-di-diffusion
CLASS_NAME = 'person' # options: person, man, woman
INSTANCE_TOKEN = '' # token that will be used in the prompts

In [ ]:
from pathlib import Path
import wget

class_path = Path('./regularization_images')
if not class_path.exists():
    class_path.mkdir(parents=True)

output_path = Path('./output').joinpath(INSTANCE_TOKEN).joinpath(MODEL_NAME)
if not output_path.exists():
    output_path.mkdir(parents=True)

CLASS_DIR = class_path.as_posix()
OUTPUT_DIR = output_path.as_posix()

if CLASS_NAME == 'person':
    data_file = 'person_v1-5_mse_vae_ddim50_cfg7_n2115'
elif CLASS_NAME == 'man':
    data_file = 'guy_v1-5_mse_vae_ddim50_cfg7_n4820'
elif CLASS_NAME == 'woman':
    data_file = 'woman_v1-5_mse_vae_ddim50_cfg7_n4420'

url = f'https://huggingface.co/datasets/ProGamerGov/StableDiffusion-v1-5-Regularization-Images/resolve/main/{data_file}.zip'

file = wget.download(url=url, out=f'{CLASS_DIR}')

filename = Path(file).name.replace('.zip', '')

!unzip -q -o -d $CLASS_DIR $file 
!rm $file

class_path = class_path.joinpath(filename)
CLASS_DIR = class_path.as_posix()

### Uploading your personal images, You have 2 options:
- Add Google Drive URL that contains your images in the url variable and run the cell.
- Upload them directly in "instance_images" directory and don't run the cell.
- Make sure all images are in PNG format.

In [ ]:
import gdown
import os

instances_path = Path('./instance_images')
if not instances_path.exists():
    instances_path.mkdir(parents=True)

INSTANCE_DIR = instances_path.as_posix()

url = ""

gdown.download_folder(url, quiet=True, use_cookies=False, output=INSTANCE_DIR)

#### Renaming all images to the given instance token

In [ ]:
for i, img in enumerate(os.listdir(INSTANCE_DIR)):
        os.rename(instances_path.joinpath(img).as_posix(), 
                        instances_path.joinpath(f'{INSTANCE_TOKEN}-({i}).png').as_posix())

### Training
- Enter the training steps (200) * n_images (eg. if n_images=10 then enter 2000)
- Enter the number of class images you want to use while training

In [ ]:
import random

TRAINING_STEPS = 4000 # it should be (200 * number of images) you uploaded
CLASS_IMAGES_NUM = 200 # the number of class images you want to use in the training
SEED = random.randint(1, 999999)

#### Start Training

In [ ]:
steps = TRAINING_STEPS // 10
!accelerate launch --mixed_precision="fp16" diffusers/examples/dreambooth/train_dreambooth.py \
    --image_captions_filename \
    --train_text_encoder \
    --dump_only_text_encoder \
    --pretrained_model_name_or_path="$MODEL_NAME" \
    --instance_data_dir="$INSTANCE_DIR" \
    --class_data_dir="$CLASS_DIR" \
    --output_dir="$OUTPUT_DIR" \
    --with_prior_preservation --prior_loss_weight=1.0 \
    --mixed_precision="fp16" \
    --seed=$SEED \
    --resolution=512 \
    --train_batch_size=1 \
    --gradient_accumulation_steps=1 --gradient_checkpointing \
    --use_8bit_adam \
    --learning_rate=2e-6 \
    --lr_scheduler="polynomial" \
    --lr_warmup_steps=0 \
    --max_train_steps=$steps \
    --num_class_images=$CLASS_IMAGES_NUM

!accelerate launch --mixed_precision="fp16" diffusers/examples/dreambooth/train_dreambooth.py \
    --image_captions_filename \
    --train_only_unet \
    --save_starting_step=5000 \
    --save_n_steps=5000 \
    --pretrained_model_name_or_path="$MODEL_NAME" \
    --instance_data_dir="$INSTANCE_DIR" \
    --output_dir="$OUTPUT_DIR" \
    --seed=$SEED \
    --resolution=512 \
    --mixed_precision="fp16" \
    --train_batch_size=1 \
    --gradient_accumulation_steps=1 --gradient_checkpointing \
    --use_8bit_adam \
    --learning_rate=2e-6 \
    --lr_scheduler="polynomial" \
    --lr_warmup_steps=0 \
    --max_train_steps=$TRAINING_STEPS

### Convert trained model to CKPT
- After you run this cell you will find your trained model inside "models" directory.
- Also it will make a copy for the model inside AUTOMATIC1111 so you can use it with the UI.

In [ ]:
ckpt_path = Path('./models')
if not ckpt_path.exists():
    ckpt_path.mkdir(parents=True)

ckpt_path = ckpt_path.joinpath(f'{INSTANCE_TOKEN}.ckpt')
CKPT_DIR = ckpt_path.as_posix()

!python diffusers/scripts/convert_diffusers_to_original_stable_diffusion.py --model_path $OUTPUT_DIR  --checkpoint_path $CKPT_DIR --half

api_ckpt_path = Path(f'../stable-diffusion-webui/models/Stable-diffusion/{INSTANCE_TOKEN}.ckpt')
API_CKPT_DIR = api_ckpt_path.as_posix()

!cp $CKPT_DIR $API_CKPT_DIR